In [1]:
import sys 

In [ ]:
cd '/srv/flash1/hagrawal9/project/habitat/habitat-api/'

In [3]:
import gzip
import json
import os
import sys
from typing import Any, Dict, List, Optional, Type
from copy import deepcopy

import attr
import cv2
import git
import magnum as mn
import numpy as np

# %matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


import habitat
import habitat_sim
from habitat.config import Config
from habitat.core.registry import registry
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations import maps, fog_of_war
from habitat.utils.visualizations.utils import observations_to_image

from rearrangement.utils.planner import (
    compute_traversable_map,
    compute_distance_using_fmm,
    find_dist_from_map,
    compute_distance_mat_using_navmesh,
    compute_distance_mat_using_fmm,
    find_shortest_path_for_multiple_objects
)
from rearrangement.utils.visualization import (
    get_top_down_map
)
from rearrangement.utils.geometry import (
     geodesic_distance
)
from collections import defaultdict

In [4]:
from habitat.datasets.rearrangement.rearrangement_dataset import RearrangementDatasetV0
from habitat.tasks.rearrangement.rearrangement_task import RearrangementEpisode

In [5]:
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
# %cd $dir_path
data_path = os.path.join(dir_path, "data")
output_directory = "data/tutorials/output/"  # @param {type:"string"}
output_path = os.path.join(dir_path, output_directory)

In [6]:
config = habitat.get_config("configs/tasks/rearrangement_debug.yaml")

In [7]:
config.defrost()
config.TASK.SENSORS =  ["GRIPPED_OBJECT_SENSOR", "ALL_OBJECT_POSITIONS", "ALL_OBJECT_GOALS"]
config.TASK.MEASUREMENTS = ['OBJECT_TO_GOAL_DISTANCE', 'AGENT_TO_OBJECT_DISTANCE', 'TOP_DOWN_MAP']
config.DATASET.SPLIT = "test"
# config.DATASET.CONTENT_SCENES = ["rearrangement_v4_val_n=10_o=-1_Willow"]
config.freeze()

In [8]:
config.TASK.TOP_DOWN_MAP.MAP_RESOLUTION

256

In [9]:
try:
    env.close()
except:
    pass
env = habitat.Env(config)
fog_of_war_mask = None
pickup_order = {}


2020-11-04 19:52:09,326 Initializing dataset RearrangementDataset-v0
2020-11-04 19:52:09,333 initializing sim RearrangementSim-v0
I1104 19:52:14.943583 54030 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Spencerville.navmesh
I1104 19:52:14.948261 54030 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.
2020-11-04 19:52:15,053 Initializing task RearrangementTask-v0


In [10]:
len(env.episodes)

10

In [11]:
def compute_oracle_pickup_order(env):
    # obs = env.reset()
    metrics = env.get_metrics()
    episode = env.current_episode
    metrics = env.get_metrics()

    agent_pos = env._sim.get_agent(0).get_state().position
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]
    
    dist_mat = compute_distance_mat_using_navmesh(env._task._simple_pathfinder, agent_pos, object_positions, goal_positions)
    route_indexes_navmesh, pickup_order_navmesh = find_shortest_path_for_multiple_objects(dist_mat)
    
#     top_down_map, fog_of_war_mask = get_top_down_map(
#         env, env._task._simple_pathfinder, ignore_objects=True, fog_of_war_mask=None, draw_fow=False, 
#         draw_agent=False, draw_object_start_pos=False, draw_object_final_pos=False, draw_object_curr_pos=False
#     )
    
    
#     a_y, a_x = maps.to_grid(
#         agent_pos[2],
#         agent_pos[0],
#         top_down_map.shape[0:2],
#         sim=env._sim,
#     )
#     grid_object_positions = []
#     grid_goal_positions = []

#     for i, obj_pos in enumerate(object_positions):
#         tdm_pos = maps.to_grid(
#             obj_pos[2],
#             obj_pos[0],
#             top_down_map.shape[0:2],
#             sim=env._sim,
#         )
#         grid_object_positions.append(tdm_pos)

#     for i, goal_pos in enumerate(goal_positions):
#         tdm_pos = maps.to_grid(
#             goal_pos[2],
#             goal_pos[0],
#             top_down_map.shape[0:2],
#             sim=env._sim,
#         )

#         grid_goal_positions.append(tdm_pos)

#     tdmap = np.copy(top_down_map[:, :, 0])
#     tdmap = tdmap / np.max(tdmap)
#     dist_mat_map = compute_distance_mat_using_fmm(tdmap, [a_y, a_x], grid_object_positions, grid_goal_positions)
    
    
#     route_indexes_map, pickup_order_map = find_shortest_path_for_multiple_objects(dist_mat_map/3)
    
    return {
        'episode_id': episode.episode_id, 
        'scene_id': episode.scene_id,
        'pickup_order': pickup_order_navmesh,
        'dist_mat': dist_mat
        # 'pickup_order_fmm': pickup_order_map 
    }

In [12]:
def compute_l2dist_pickup_order(env):
    # obs = env.reset()
    metrics = env.get_metrics()
    episode = env.current_episode
    metrics = env.get_metrics()

    agent_pos = env._sim.get_agent(0).get_state().position
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]
    
    dist_mat = np.zeros((   
            1 + len(object_positions) + len(goal_positions), 
            1 + len(object_positions) + len(goal_positions)
    ))
    
    for i, object_pos in enumerate([agent_pos] + object_positions + goal_positions):
        for j, goal_pos in enumerate([agent_pos] + object_positions + goal_positions):
            if j == 0:  # distance from object / goal position -> "depot" is zero
                dist = 0
            else:
                dist = np.linalg.norm(
                    np.array(object_pos) - np.array(goal_pos), ord=2
                )
            dist_mat[i][j] = dist
    
    route_indexes_map, pickup_order_map = find_shortest_path_for_multiple_objects(dist_mat)
    
    return {
        'episode_id': episode.episode_id, 
        'scene_id': episode.scene_id,
#         'pickup_order': pickup_order_navmesh,
        'pickup_order_l2dist': pickup_order_map 
    }

In [13]:
def start_env_episode_distance(env, task, episode, pickup_order):
    pathfinder = env._sim.pathfinder
    agent_pos = episode.start_position
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]

    prev_obj_end_pos = agent_pos
    shortest_dist = 0

    for i in range(len(pickup_order)):
        curr_idx = pickup_order[i] - 1
        curr_obj_start_pos = object_positions[curr_idx]
        curr_obj_end_pos = goal_positions[curr_idx]
        shortest_dist += geodesic_distance(
                pathfinder, prev_obj_end_pos, [curr_obj_start_pos]
            )

        shortest_dist += geodesic_distance(
                    pathfinder, curr_obj_start_pos, [curr_obj_end_pos]
                )
        prev_obj_end_pos = curr_obj_end_pos

    return shortest_dist

In [14]:
# When pickup order is not known! 

l2dists = []
odists = []
l2pos = []
opos = []
ids = []
pod = {}
l2pod = {}

count = 0 
overall_count = 0
for episode in env.episodes: 
    obs = env.reset()
    odata = compute_oracle_pickup_order(env)
    
    l2data = compute_l2dist_pickup_order(env)
    
    pod[odata['episode_id'] + '_' + odata['scene_id']] = odata
    l2pod[l2data['episode_id'] + '_' + l2data['scene_id']] = l2data
    
    opo = odata['pickup_order']
    l2po = l2data['pickup_order_l2dist']
    opos.append(opo)
    l2pos.append(l2po)
    
    odist = start_env_episode_distance(env, env._task, env.current_episode, opo)
    l2dist = start_env_episode_distance(env, env._task, env.current_episode, l2po)
    
    odists.append(odist)
    l2dists.append(l2dist)
    
    ids.append(env.current_episode.scene_id + '_' + str(env.current_episode.episode_id))
    
    if(overall_count % 10 == 0):
        print(overall_count, sum(odists) / sum(l2dists))
        
    if l2dist > odist:
        
        count += 1
        # print(count, overall_count, float(count/overall_count))
        
    overall_count +=1

0 1.0103155174669216


In [17]:
count, overall_count, sum(odists)/sum(l2dists)

(6, 10, 0.9969686382327528)

In [31]:
l2dists = []
odists = []
l2pos = []
opos = []
ids = []
pod = {}
l2pod = {}

count = 0 
overall_count = 0
for ep in env.episodes: 
    obs = env.reset()
    episode = env.current_episode
    
    opo = episode.pickup_order
    l2po = episode.pickup_order_l2dist
    
    opos.append(opo)
    l2pos.append(l2po)
    
    odist = start_env_episode_distance(env, env._task, env.current_episode, opo)
    l2dist = start_env_episode_distance(env, env._task, env.current_episode, l2po)
    
    odists.append(odist)
    l2dists.append(l2dist)
    
    ids.append(env.current_episode.scene_id + '_' + str(env.current_episode.episode_id))
    
    if(overall_count % 10 == 0):
        print(overall_count, sum(odists) / sum(l2dists))
    
    overall_count +=1

TypeError: object of type 'NoneType' has no len()

In [84]:
while(len(pickup_order)!=len(env.episodes)):
    data = compute_oracle_pickup_order(env)
    pickup_order[data['episode_id'] + '_' + data['scene_id']] = data
    print('\rEpisode ID: {} Done: {} '.format(data['episode_id'], len(pickup_order)), end=" ")

Episode ID: 8 Done: 10  

I1030 06:38:43.626117 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Sasakwa.navmesh
I1030 06:38:43.627959 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 7 Done: 20  

I1030 06:39:00.113559 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Eagerville.navmesh
I1030 06:39:00.116208 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 2 Done: 30  

...

Episode ID: 0 Done: 79  

I1030 06:40:38.543604 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Kemblesville.navmesh
I1030 06:40:38.545637 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 2 Done: 89  

I1030 06:40:53.617020 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Willow.navmesh
I1030 06:40:53.619477 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 8 Done: 100  

In [93]:
while(len(pickup_order)!=len(env.episodes)):
    data = compute_l2dist_pickup_order(env)
    pickup_order[data['episode_id'] + '_' + data['scene_id']] = data
    print('\rEpisode ID: {} Done: {} '.format(data['episode_id'], len(pickup_order)), end=" ")

Episode ID: 0 Done: 9  

I1030 10:23:23.899402 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Sasakwa.navmesh
I1030 10:23:23.901108 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 4 Done: 19  

I1030 10:23:40.001954 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Eagerville.navmesh
I1030 10:23:40.004197 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 6 Done: 29  

...

Episode ID: 2 Done: 89  

I1030 10:25:32.636682 34157 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Willow.navmesh


Episode ID: 4 Done: 90  

I1030 10:25:32.640164 34157 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.


Episode ID: 8 Done: 100  

In [94]:
with open('data/pickup_order_l2dist_val_v4.json','w') as f:
    json.dump(pickup_order, f)

In [95]:
with open('data/pickup_order_l2dist_val_v4.json','r') as f:
    pickup_order_l2dist = json.load(f)
    
with open('data/pickup_order_oracle_val_v4.json','r') as f:
    pickup_order_tdmap = json.load(f)

In [107]:
updated_scene_data = []
split = 'val'

for filename in os.listdir('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split)):
    scene_id = filename.split('.')[0].split('_')[-1]
    with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "rt") as f:
        scene_data = json.load(f)
    updated_scene_data = deepcopy(scene_data)
    
    for i, episode in enumerate(scene_data['episodes']):
        episode_id = episode['episode_id']
        assert updated_scene_data['episodes'][i]['episode_id'] == episode['episode_id']
        assert updated_scene_data['episodes'][i]['scene_id'] == episode['scene_id']
        updated_scene_data['episodes'][i]['pickup_order'] = pickup_order_tdmap[str(episode['episode_id']) + "_" + episode['scene_id']]['pickup_order']
        updated_scene_data['episodes'][i]['pickup_order_l2dist'] = pickup_order_l2dist[str(episode['episode_id']) + "_" + episode['scene_id']]['pickup_order_l2dist']
    
    
    with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "wt") as f:
        json.dump(updated_scene_data, f)
        
    print(filename)

rearrangement_v4_val_n=10_o=-1_Kemblesville.json.gz
rearrangement_v4_val_n=10_o=-1_Hallettsville.json.gz
rearrangement_v4_val_n=10_o=-1_Sodaville.json.gz
rearrangement_v4_val_n=10_o=-1_Castor.json.gz
rearrangement_v4_val_n=10_o=-1_Eagerville.json.gz
rearrangement_v4_val_n=10_o=-1_Model.json.gz
rearrangement_v4_val_n=10_o=-1_Willow.json.gz
rearrangement_v4_val_n=10_o=-1_Sasakwa.json.gz
rearrangement_v4_val_n=10_o=-1_Chrisney.json.gz
rearrangement_v4_val_n=10_o=-1_Maugansville.json.gz


In [97]:
pwd

'/coc/testnvme/hagrawal9/project/habitat/habitat-api'

In [100]:
len(pickup_order_l2dist), len(pickup_order_tdmap)

(100, 100)